# Préparation des variables de mode de vie

In [ ]:
library(tidyverse)
library(factoextra)

# Charger les données
df_pilot1_2022 <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/01_pilot1_2022.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_mode_de_vie <- df_pilot1_2022 %>%
  select(all_of(variables_mode_de_vie))

In [ ]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("vehicule")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))

df_mode_de_vie %>%
  select(starts_with("act_transport")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))

In [ ]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

### Transformations

- Regroupement de ses_dwelling
  - ses_dwelling_Condo : ses_dwelling_Condo + ses_dwelling_Loft + ses_dwelling_Tour
  - ses_dwelling_Other : ses_dwelling_Other + ses_dwelling_Coop + ses_dwelling_HLM + ses_dwelling_Mobile

- Regroupement des vehicules
  - vehicule_ToutTerrain : vehicule_4x4	+ vehicule_PickUp
  - vehicle_Voiture : vehicule_Berline + vehicule_Cabriolet + vehicule_luxury + vehicule_sport
  - vehicule_Van
  - vehicule_electric
  - vehicule_VUS
  - vehicule_other
  - vehicule_noCar

- Regroupement des act_transport
  - act_transport_Car : act_transport_Car + act_transport_Moto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - Les autres ne sont pas regroupés

In [ ]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwelling_Condo = if_else(
      ses_dwelling_Loft == 1 | ses_dwelling_Tour == 1,
      1,
      ses_dwelling_Condo
    ),
    ses_dwelling_Other = if_else(
      ses_dwelling_Coop == 1 | ses_dwelling_HLM == 1 | ses_dwelling_Mobile == 1,
      1,
      ses_dwelling_Other
    )
  ) %>%
  select(
    -ses_dwelling_Loft,
    -ses_dwelling_Tour,
    -ses_dwelling_Coop,
    -ses_dwelling_HLM,
    -ses_dwelling_Mobile
  )

In [ ]:
# Regroupement de vehicule
#-------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    vehicule_ToutTerrain = if_else(
      vehicule_4x4 == 1 | vehicule_PickUp == 1,
      1,
      0
    ),
    vehicule_Voiture = if_else(
      vehicule_Berline == 1 | vehicule_Cabriolet == 1 | vehicule_luxury == 1 | vehicule_sport,
      1,
      0
    )
  ) %>%
  select(
    -vehicule_4x4,
    -vehicule_PickUp,
    -vehicule_Berline,
    -vehicule_Cabriolet,
    -vehicule_luxury,
    -vehicule_sport
  )

In [ ]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    act_transport_Car = if_else(
      act_transport_Moto == 1,
      1,
      act_transport_Car
    )) %>%
  select(
    -act_transport_Moto
  )

In [ ]:
variables_mode_de_vie_clust <- c(
  "id",
  #"postal_code", # À transformer en rural, urbain, région, banlieue
  "ses_dwelling_App",
  "ses_dwelling_Condo",
  "ses_dwelling_detachedHouse",
  "ses_dwelling_townHouse",
  "ses_dwelling_Other",
  "act_transport_Car",
  "act_transport_SUV",
  "act_transport_Walk",
  "act_transport_Bicycle",
  "act_transport_PublicTransportation",
  "vehicule_ToutTerrain",
  "vehicule_Van",
  "vehicule_Voiture",
  "vehicule_electric",
  "vehicule_VUS",
  "vehicule_other",
  "vehicule_noCar"
)

In [ ]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

In [ ]:
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/02_pilot1_2022_mode_de_vie.rds")

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.